In [1]:
import torch

In [2]:
dim = 2048 // 8
max_seq_len = 2048
theta = 10000
freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
t = torch.arange(max_seq_len * 2, device=freqs.device, dtype=torch.float32)

In [2]:
import torch
mask = torch.full((1, 1, 1, 1), float("-inf"))
mask = mask.to(torch.float32).triu(diagonal=1)

In [4]:
seqlen = 5
x = torch.rand(3, 3).to('cuda:1').half()

mask1 = torch.full(
    (1, 1, seqlen, seqlen), float("-inf"), device=x.device
).to(torch.float32).triu(diagonal=1).type_as(x)
mask2 = torch.full(
    (1, 1, seqlen, seqlen), float("-inf"), device=x.device, dtype=x.dtype
).triu(diagonal=1)

In [12]:
torch.tensor([3.14, 10.]).cos()

tensor([-1.0000, -0.8391])

In [7]:
x = torch.rand(3, 2)
y = torch.rand(2, 3).half()
torch.matmul(x, y)

RuntimeError: expected m1 and m2 to have the same dtype, but got: float != c10::Half

In [10]:
1 / theta ** torch.arange(0, dim, 2)[:(dim // 2)].float() / dim

tensor([4.8828e-04, 4.8828e-12, 4.8828e-20,  ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00])